<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_2_Memory_Basics_Teaching_Agents_to_Remember.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial 2: Memory Basics - Teaching Agents to Remember

 In this tutorial, we'll add memory capabilities to our agents so they can remember previous conversations and build context over time.

 What we'll learn:
 - Working memory for short-term context
 - Memory storage and retrieval patterns
 - Context-aware response generation
 - Memory cleanup and capacity management

 This builds directly on Tutorial 1's foundation!

In [12]:
import uuid
import time
import threading
import queue
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple
from enum import Enum
import json

In [3]:
# Import our foundation from Tutorial 1
AgentID = str
MessageID = str

class MessagePriority(Enum):
    LOW = 1
    NORMAL = 2
    HIGH = 3
    URGENT = 4

@dataclass
class Message:
    id: MessageID
    sender: AgentID
    recipients: List[AgentID]
    content: Any
    priority: MessagePriority = MessagePriority.NORMAL
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: float = field(default_factory=time.time)

    @classmethod
    def create(cls, sender: AgentID, recipients: List[AgentID], content: Any,
               priority: MessagePriority = MessagePriority.NORMAL) -> 'Message':
        return cls(
            id=str(uuid.uuid4()),
            sender=sender,
            recipients=recipients,
            content=content,
            priority=priority
        )

class AgentMetrics:
    def __init__(self):
        self.message_count = 0
        self.processing_time = 0.0
        self.error_count = 0
        self.last_active = None
        # New memory-related metrics
        self.memory_operations = 0
        self.memory_hits = 0
        self.memory_misses = 0

    def update_processing_time(self, time_delta: float):
        self.processing_time += time_delta
        self.last_active = time.time()

    def increment_message_count(self):
        self.message_count += 1
        self.last_active = time.time()

    def record_memory_operation(self, hit: bool = True):
        """Record a memory access operation"""
        self.memory_operations += 1
        if hit:
            self.memory_hits += 1
        else:
            self.memory_misses += 1

class MemoryType(Enum):
    """
    Different types of memory for different purposes

    Working memory is like your conscious attention - limited capacity,
    immediate access, temporary storage for active thinking.
    """
    WORKING = "working"      # Short-term, limited capacity, immediate access
    EPISODIC = "episodic"    # Event-based memories (what happened when)
    SEMANTIC = "semantic"    # Fact-based memories (what is true)

@dataclass
class MemoryItem:
    """
    A single item stored in agent memory

    This represents one piece of information the agent remembers.
    It could be a conversation snippet, a learned fact, or context.
    """
    id: str                                    # Unique identifier for this memory
    content: Any                              # What is being remembered
    memory_type: MemoryType                   # What kind of memory this is
    created_at: float = field(default_factory=time.time)  # When was this stored
    last_accessed: float = field(default_factory=time.time)  # When was this last used
    access_count: int = 0                     # How many times has this been accessed
    importance: float = 1.0                   # How important is this memory (0.0 to 1.0)
    metadata: Dict[str, Any] = field(default_factory=dict)  # Extra information

    def access(self):
        """Mark this memory as accessed - updates statistics"""
        self.last_accessed = time.time()
        self.access_count += 1

    def age_in_seconds(self) -> float:
        """How old is this memory in seconds"""
        return time.time() - self.created_at

    def time_since_last_access(self) -> float:
        """How long since this memory was last accessed"""
        return time.time() - self.last_accessed

class WorkingMemory:
    """
    Working memory system for agents

    This is like short-term memory or conscious attention. It has limited
    capacity and is used for immediate context and active processing.

    Think of it like the "recent messages" you keep in mind during a conversation.
    """

    def __init__(self, capacity: int = 10):
        self.capacity = capacity                    # Maximum number of items to store
        self._items: List[MemoryItem] = []         # Ordered list of memory items
        self._access_lock = threading.Lock()       # Thread safety for memory operations

        print(f"🧠 Initialized working memory with capacity {capacity}")

    def store(self, content: Any, importance: float = 1.0, metadata: Dict[str, Any] = None) -> str:
        """
        Store an item in working memory

        If we're at capacity, the least important and oldest items get removed.
        This mimics how human working memory works - we can only hold so much.
        """
        with self._access_lock:
            # Create the memory item
            memory_item = MemoryItem(
                id=str(uuid.uuid4()),
                content=content,
                memory_type=MemoryType.WORKING,
                importance=importance,
                metadata=metadata or {}
            )

            # Add to the front of the list (most recent first)
            self._items.insert(0, memory_item)

            # If we exceed capacity, remove the least important old items
            if len(self._items) > self.capacity:
                # Sort by importance (descending) then by age (oldest first for same importance)
                self._items.sort(key=lambda x: (-x.importance, x.created_at))
                # Keep only the top items
                removed_items = self._items[self.capacity:]
                self._items = self._items[:self.capacity]

                print(f"🧠 Working memory at capacity, removed {len(removed_items)} items")

            print(f"🧠 Stored in working memory: {str(content)[:50]}{'...' if len(str(content)) > 50 else ''}")
            return memory_item.id

    def retrieve(self, query: str = None, limit: int = None) -> List[MemoryItem]:
        """
        Retrieve items from working memory

        If no query is provided, returns recent items.
        If query is provided, does simple text matching.
        """
        with self._access_lock:
            if not query:
                # Return most recent items
                items = self._items[:limit] if limit else self._items
            else:
                # Simple text search in content
                items = []
                for item in self._items:
                    if query.lower() in str(item.content).lower():
                        items.append(item)
                        if limit and len(items) >= limit:
                            break

            # Mark retrieved items as accessed
            for item in items:
                item.access()

            print(f"🧠 Retrieved {len(items)} items from working memory")
            return items

    def get_context(self, max_items: int = 5) -> List[Any]:
        """
        Get recent context for conversation

        This is commonly used to provide context to agents about
        recent conversation history.
        """
        recent_items = self.retrieve(limit=max_items)
        context = [item.content for item in recent_items]

        if context:
            print(f"🧠 Providing {len(context)} items as context")

        return context

    def clear(self):
        """Clear all working memory"""
        with self._access_lock:
            cleared_count = len(self._items)
            self._items.clear()
            print(f"🧠 Cleared {cleared_count} items from working memory")

    def size(self) -> int:
        """Get current number of items in memory"""
        return len(self._items)

    def get_stats(self) -> Dict[str, Any]:
        """Get statistics about working memory usage"""
        with self._access_lock:
            if not self._items:
                return {
                    "size": 0,
                    "capacity": self.capacity,
                    "utilization": 0.0,
                    "average_age": 0.0,
                    "average_importance": 0.0
                }

            current_time = time.time()
            ages = [current_time - item.created_at for item in self._items]
            importances = [item.importance for item in self._items]

            return {
                "size": len(self._items),
                "capacity": self.capacity,
                "utilization": len(self._items) / self.capacity,
                "average_age": sum(ages) / len(ages),
                "average_importance": sum(importances) / len(importances),
                "oldest_item_age": max(ages) if ages else 0.0,
                "total_accesses": sum(item.access_count for item in self._items)
            }

class BaseAgent(ABC):
    """Enhanced BaseAgent with working memory capabilities"""

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 memory_capacity: int = 10):
        # Core identification
        self.id = agent_id or str(uuid.uuid4())
        self.name = name or self.__class__.__name__

        # Message processing infrastructure
        self._message_queue = queue.Queue()
        self._stop_event = threading.Event()
        self._processing_thread = None

        # Memory system - NEW!
        self.working_memory = WorkingMemory(capacity=memory_capacity)

        # Monitoring and metrics
        self._metrics = AgentMetrics()
        self._running = False

        print(f"🤖 Initialized agent: {self.name} ({self.id[:8]}...) with {memory_capacity} memory slots")

    def start(self):
        if self._running:
            print(f"⚠️  Agent {self.name} is already running")
            return

        self._stop_event.clear()
        self._processing_thread = threading.Thread(
            target=self._processing_loop,
            daemon=True,
            name=f"Agent-{self.name}"
        )
        self._processing_thread.start()
        self._running = True
        print(f"▶️  Agent {self.name} started")

    def stop(self):
        if not self._running:
            return

        self._stop_event.set()

        if self._processing_thread:
            self._processing_thread.join(timeout=2.0)
            if self._processing_thread.is_alive():
                print(f"⚠️  Warning: Agent {self.name} thread didn't stop cleanly")

        self._running = False
        print(f"⏹️  Agent {self.name} stopped")

    def receive_message(self, message: Message):
        self._message_queue.put(message)
        print(f"📬 Agent {self.name} received message: {str(message.content)[:50]}{'...' if len(str(message.content)) > 50 else ''}")

    def send_message(self, recipients: List[AgentID], content: Any,
                    priority: MessagePriority = MessagePriority.NORMAL) -> Message:
        message = Message.create(
            sender=self.id,
            recipients=recipients,
            content=content,
            priority=priority
        )

        self._metrics.increment_message_count()
        print(f"📤 Agent {self.name} sent message: {str(content)[:50]}{'...' if len(str(content)) > 50 else ''}")

        return message

    def remember(self, content: Any, importance: float = 1.0, metadata: Dict[str, Any] = None) -> str:
        """
        Store something in working memory

        This is a convenience method that agents can use to remember
        important information from their processing.
        """
        memory_id = self.working_memory.store(content, importance, metadata)
        self._metrics.record_memory_operation(hit=True)
        return memory_id

    def recall(self, query: str = None, limit: int = None) -> List[MemoryItem]:
        """
        Retrieve items from working memory

        This is how agents can access their remembered information.
        """
        items = self.working_memory.retrieve(query, limit)
        hit = len(items) > 0
        self._metrics.record_memory_operation(hit=hit)
        return items

    def get_conversation_context(self, max_items: int = 5) -> List[Any]:
        """
        Get recent conversation context

        This helps agents maintain context across multiple messages.
        """
        return self.working_memory.get_context(max_items)

    @abstractmethod
    def process_message(self, message: Message):
        pass

    def _processing_loop(self):
        print(f"🔄 Agent {self.name} processing loop started")

        while not self._stop_event.is_set():
            try:
                message = self._message_queue.get(timeout=0.1)

                start_time = time.time()
                self.process_message(message)
                processing_time = time.time() - start_time

                self._metrics.update_processing_time(processing_time)
                self._message_queue.task_done()

                print(f"✅ Agent {self.name} processed message in {processing_time:.3f}s")

            except queue.Empty:
                continue
            except Exception as e:
                print(f"❌ Error processing message in {self.name}: {e}")
                self._metrics.error_count += 1

        print(f"🛑 Agent {self.name} processing loop stopped")

    def get_metrics(self) -> Dict[str, Any]:
        base_metrics = {
            "message_count": self._metrics.message_count,
            "processing_time": self._metrics.processing_time,
            "error_count": self._metrics.error_count,
            "last_active": self._metrics.last_active,
            "average_processing_time": (
                self._metrics.processing_time / max(self._metrics.message_count, 1)
            ),
            # New memory metrics
            "memory_operations": self._metrics.memory_operations,
            "memory_hits": self._metrics.memory_hits,
            "memory_misses": self._metrics.memory_misses,
            "memory_hit_rate": (
                self._metrics.memory_hits / max(self._metrics.memory_operations, 1)
            )
        }

        # Add memory system statistics
        base_metrics["memory_stats"] = self.working_memory.get_stats()

        return base_metrics

class ConversationAgent(BaseAgent):
    """
    An agent that remembers previous conversations and provides context-aware responses

    This agent demonstrates how to use working memory to maintain conversation
    context. It remembers previous messages and can refer back to them.
    """

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 memory_capacity: int = 15):
        super().__init__(agent_id, name, memory_capacity)
        self.conversation_turns = 0  # Track how many turns we've had

    def process_message(self, message: Message):
        """
        Process messages with conversation memory

        This implementation:
        1. Remembers the incoming message
        2. Retrieves relevant context from memory
        3. Generates a context-aware response
        4. Remembers the response for future context
        """
        print(f"🎯 ConversationAgent {self.name} processing message from {message.sender}")
        print(f"   Incoming: '{message.content}'")

        # Remember this incoming message
        self.remember(
            content=f"User said: {message.content}",
            importance=1.0,
            metadata={
                "type": "incoming_message",
                "sender": message.sender,
                "turn": self.conversation_turns
            }
        )

        # Get conversation context to inform our response
        context = self.get_conversation_context(max_items=5)

        # Generate a context-aware response
        response_content = self._generate_response(message.content, context)

        # Remember our response
        self.remember(
            content=f"I responded: {response_content}",
            importance=0.8,
            metadata={
                "type": "outgoing_message",
                "recipient": message.sender,
                "turn": self.conversation_turns
            }
        )

        # Send the response
        response = self.send_message(
            recipients=[message.sender],
            content=response_content,
            priority=MessagePriority.NORMAL
        )

        self.conversation_turns += 1
        print(f"   Response: '{response.content}'")
        print(f"   (Conversation turn #{self.conversation_turns})")

    def _generate_response(self, user_input: str, context: List[Any]) -> str:
        """
        Generate a response based on user input and conversation context

        This is where the magic happens - using memory to create
        contextually appropriate responses.
        """
        user_lower = user_input.lower()

        # Check if user is referring to previous conversation
        if any(word in user_lower for word in ["before", "earlier", "previous", "last", "remember"]):
            if context:
                return f"Yes, I remember our conversation! We were talking about: {context[0] if context else 'nothing specific yet'}"
            else:
                return "I don't have any previous conversation to reference yet."

        # Check if user is asking about memory
        elif any(word in user_lower for word in ["memory", "remember", "forget", "recall"]):
            memory_stats = self.working_memory.get_stats()
            return f"I currently remember {memory_stats['size']} things from our conversation. My memory is {memory_stats['utilization']:.1%} full."

        # Greetings
        elif any(word in user_lower for word in ["hello", "hi", "hey", "greetings"]):
            if self.conversation_turns > 0:
                return f"Hello again! This is the {self.conversation_turns + 1} thing you've said to me."
            else:
                return "Hello! This is the start of our conversation. I'll remember what we talk about!"

        # Farewells
        elif any(word in user_lower for word in ["bye", "goodbye", "farewell", "see you"]):
            memory_stats = self.working_memory.get_stats()
            return f"Goodbye! I'll remember our conversation of {memory_stats['size']} exchanges. It was nice talking with you!"

        # Questions
        elif "?" in user_input:
            if context:
                recent_topics = [str(item)[:30] for item in context[:3]]
                return f"That's an interesting question! Based on our conversation about {', '.join(recent_topics)}, I think you're asking something thoughtful."
            else:
                return "That's a great question! I'm storing it in my memory for context."

        # Default response with context awareness
        else:
            if context and len(context) > 2:
                return f"I see we're building up quite a conversation! You've mentioned several things, and I'm keeping track of it all."
            elif context:
                return f"Interesting! That adds to what we were discussing before."
            else:
                return f"Thank you for starting our conversation! I'll remember this: '{user_input}'"


In [4]:

# =============================================================================
# DEMO SECTION: Let's test our memory-enabled agent!
# =============================================================================

print("=" * 60)
print("🚀 Tutorial 2: Memory Basics - Teaching Agents to Remember")
print("=" * 60)
print()

# Step 1: Create a ConversationAgent with memory
print("📝 Step 1: Creating a ConversationAgent with memory...")
memory_agent = ConversationAgent(name="MemoryBot", memory_capacity=10)
print()

# Step 2: Start the agent
print("📝 Step 2: Starting the memory-enabled agent...")
memory_agent.start()
print()

# Step 3: Have a multi-turn conversation to test memory
print("📝 Step 3: Having a conversation to test memory...")
print()

conversation = [
    "Hello, my name is Alice",
    "I'm working on a Python project",
    "Do you remember my name?",
    "What was I working on?",
    "How much do you remember about our chat?",
    "Goodbye!"
]

for i, message_text in enumerate(conversation, 1):
    print(f"--- Conversation Turn {i} ---")

    # Create and send message
    test_message = Message.create(
        sender="alice_user",
        recipients=[memory_agent.id],
        content=message_text
    )

    print(f"🗣️  Alice says: '{message_text}'")
    memory_agent.receive_message(test_message)

    # Wait for processing
    time.sleep(0.3)
    print()

# Step 4: Check the agent's memory state
print("📝 Step 4: Examining the agent's memory...")
memory_stats = memory_agent.working_memory.get_stats()
print("   Memory Statistics:")
for key, value in memory_stats.items():
    if isinstance(value, float):
        print(f"     {key}: {value:.3f}")
    else:
        print(f"     {key}: {value}")
print()

# Step 5: Retrieve specific memories
print("📝 Step 5: Testing memory retrieval...")
name_memories = memory_agent.recall(query="Alice", limit=3)
print(f"   Found {len(name_memories)} memories about 'Alice':")
for memory in name_memories:
    print(f"     - {memory.content}")
print()

project_memories = memory_agent.recall(query="Python", limit=3)
print(f"   Found {len(project_memories)} memories about 'Python':")
for memory in project_memories:
    print(f"     - {memory.content}")
print()

# Step 6: Check comprehensive metrics
print("📝 Step 6: Checking comprehensive agent metrics...")
metrics = memory_agent.get_metrics()
print("   Agent Performance:")
for key, value in metrics.items():
    if key == 'memory_stats':
        print(f"     {key}:")
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, float):
                print(f"       {sub_key}: {sub_value:.3f}")
            else:
                print(f"       {sub_key}: {sub_value}")
    elif key == 'last_active' and value:
        last_active_str = time.strftime('%H:%M:%S', time.localtime(value))
        print(f"     {key}: {last_active_str}")
    elif isinstance(value, float):
        print(f"     {key}: {value:.3f}")
    else:
        print(f"     {key}: {value}")
print()

# Step 7: Test memory capacity limits
print("📝 Step 7: Testing memory capacity limits...")
print("   Filling memory beyond capacity to test cleanup...")

for i in range(15):  # More than our capacity of 10
    memory_agent.remember(
        content=f"Test memory item #{i}",
        importance=0.5,
        metadata={"test": True, "sequence": i}
    )

final_stats = memory_agent.working_memory.get_stats()
print(f"   After adding 15 items to capacity-10 memory:")
print(f"     Current size: {final_stats['size']}")
print(f"     Utilization: {final_stats['utilization']:.1%}")
print()

# Step 8: Clean up
print("📝 Step 8: Stopping the agent and cleaning up...")
memory_agent.stop()
print()

print("✅ Tutorial 2 Complete!")
print()


🚀 Tutorial 2: Memory Basics - Teaching Agents to Remember

📝 Step 1: Creating a ConversationAgent with memory...
🧠 Initialized working memory with capacity 10
🤖 Initialized agent: MemoryBot (300e74f1...) with 10 memory slots

📝 Step 2: Starting the memory-enabled agent...
🔄 Agent MemoryBot processing loop started
▶️  Agent MemoryBot started

📝 Step 3: Having a conversation to test memory...

--- Conversation Turn 1 ---
🗣️  Alice says: 'Hello, my name is Alice'
📬 Agent MemoryBot received message: Hello, my name is Alice
🎯 ConversationAgent MemoryBot processing message from alice_user
   Incoming: 'Hello, my name is Alice'
🧠 Stored in working memory: User said: Hello, my name is Alice
🧠 Retrieved 1 items from working memory
🧠 Providing 1 items as context
🧠 Stored in working memory: I responded: Hello! This is the start of our conve...
📤 Agent MemoryBot sent message: Hello! This is the start of our conversation. I'll...
   Response: 'Hello! This is the start of our conversation. I'll reme

In [11]:
# Collect memory data for visualization
all_memories = memory_agent.recall()  # Get all memories
if all_memories:
    # Prepare data for visualization
    memory_data = []
    for memory in all_memories:
        memory_data.append({
            'id': memory.id[:8],
            'content': str(memory.content)[:40] + ('...' if len(str(memory.content)) > 40 else ''),
            'created_at': memory.created_at,
            'last_accessed': memory.last_accessed,
            'access_count': memory.access_count,
            'importance': memory.importance,
            'age_seconds': memory.age_in_seconds(),
            'type': memory.metadata.get('type', 'unknown')
        })

    # Try to create visualization (optional - graceful fallback if plotly not available)
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        import plotly.express as px

        # Create subplot with secondary y-axis
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Memory Timeline', 'Memory Access Patterns',
                'Memory Importance Distribution', 'Memory Age vs Access Count'
            ),
            specs=[[{"secondary_y": True}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )

        # Timeline of memory creation and access
        creation_times = [time.strftime('%H:%M:%S', time.localtime(m['created_at'])) for m in memory_data]
        access_times = [time.strftime('%H:%M:%S', time.localtime(m['last_accessed'])) for m in memory_data]

        fig.add_trace(
            go.Scatter(
                x=creation_times,
                y=[m['importance'] for m in memory_data],
                mode='markers+lines',
                name='Memory Creation',
                text=[m['content'] for m in memory_data],
                marker=dict(size=10, color='blue'),
                hovertemplate='<b>%{text}</b><br>Importance: %{y}<br>Created: %{x}<extra></extra>'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=access_times,
                y=[m['access_count'] for m in memory_data],
                mode='markers',
                name='Memory Access',
                text=[m['content'] for m in memory_data],
                marker=dict(size=8, color='red', symbol='diamond'),
                hovertemplate='<b>%{text}</b><br>Access Count: %{y}<br>Last Access: %{x}<extra></extra>',
                yaxis='y2'
            ),
            row=1, col=1, secondary_y=True
        )

        # Memory access patterns
        access_counts = [m['access_count'] for m in memory_data]
        fig.add_trace(
            go.Bar(
                x=[m['id'] for m in memory_data],
                y=access_counts,
                name='Access Count',
                text=[m['content'] for m in memory_data],
                marker_color='lightblue',
                hovertemplate='<b>%{text}</b><br>Access Count: %{y}<br>ID: %{x}<extra></extra>'
            ),
            row=1, col=2
        )

        # Importance distribution
        importance_values = [m['importance'] for m in memory_data]
        fig.add_trace(
            go.Histogram(
                x=importance_values,
                nbinsx=10,
                name='Importance Distribution',
                marker_color='green',
                opacity=0.7
            ),
            row=2, col=1
        )

        # Age vs Access Count scatter
        fig.add_trace(
            go.Scatter(
                x=[m['age_seconds'] for m in memory_data],
                y=[m['access_count'] for m in memory_data],
                mode='markers',
                name='Age vs Access',
                text=[m['content'] for m in memory_data],
                marker=dict(
                    size=[m['importance'] * 20 for m in memory_data],
                    color=[m['importance'] for m in memory_data],
                    colorscale='viridis',
                    showscale=True,
                    colorbar=dict(title="Importance")
                ),
                hovertemplate='<b>%{text}</b><br>Age: %{x:.1f}s<br>Access Count: %{y}<extra></extra>'
            ),
            row=2, col=2
        )

        # Update layout
        fig.update_layout(
            title_text=f"Memory Analysis for {memory_agent.name}",
            showlegend=True,
            height=800,
            template='plotly_white'
        )

        # Update axes labels
        fig.update_xaxes(title_text="Time", row=1, col=1)
        fig.update_yaxes(title_text="Importance", row=1, col=1)
        fig.update_yaxes(title_text="Access Count", row=1, col=1, secondary_y=True)

        fig.update_xaxes(title_text="Memory ID", row=1, col=2)
        fig.update_yaxes(title_text="Access Count", row=1, col=2)

        fig.update_xaxes(title_text="Importance Value", row=2, col=1)
        fig.update_yaxes(title_text="Frequency", row=2, col=1)

        fig.update_xaxes(title_text="Age (seconds)", row=2, col=2)
        fig.update_yaxes(title_text="Access Count", row=2, col=2)

        # Show the plot
        fig.show()

        print("   ✅ Interactive memory visualization created!")
        print("   📊 The plot shows:")
        print("      - Timeline of memory creation and access patterns")
        print("      - Access frequency for each memory item")
        print("      - Distribution of memory importance values")
        print("      - Relationship between memory age and access count")
        print()

        # Create a simple memory heatmap
        fig2 = go.Figure(data=go.Heatmap(
            z=[[m['importance'], m['access_count'], m['age_seconds']] for m in memory_data],
            x=['Importance', 'Access Count', 'Age (seconds)'],
            y=[m['id'] for m in memory_data],
            colorscale='RdYlBu_r',
            text=[[f"{m['importance']:.2f}", f"{m['access_count']}", f"{m['age_seconds']:.1f}"] for m in memory_data],
            texttemplate="%{text}",
            textfont={"size": 10},
            hovertemplate='Memory: %{y}<br>Metric: %{x}<br>Value: %{z}<br>Content: %{customdata}<extra></extra>',
            customdata=[m['content'] for m in memory_data]
        ))

        fig2.update_layout(
            title=f'Memory Metrics Heatmap - {memory_agent.name}',
            xaxis_title='Metrics',
            yaxis_title='Memory Items',
            template='plotly_white'
        )

        fig2.show()
        print("   ✅ Memory metrics heatmap created!")
        print()

    except ImportError:
        print("   ⚠️  Plotly not available - skipping visualization")
        print("   💡 To see memory visualizations, install plotly: pip install plotly")
        print("   📊 Memory data summary:")
        for i, memory in enumerate(memory_data[:5]):  # Show first 5
            print(f"      Memory {i+1}: {memory['content']}")
            print(f"        Importance: {memory['importance']:.2f}, Access Count: {memory['access_count']}")
        print()
else:
    print("   ⚠️  No memories to visualize")
    print()

# Step 9: Clean up
print("📝 Step 9: Stopping the agent and cleaning up...")
memory_agent.stop()
print()

print("✅ Tutorial 2 Complete!")
print()

🧠 Retrieved 10 items from working memory


   ✅ Interactive memory visualization created!
   📊 The plot shows:
      - Timeline of memory creation and access patterns
      - Access frequency for each memory item
      - Distribution of memory importance values
      - Relationship between memory age and access count



   ✅ Memory metrics heatmap created!

📝 Step 9: Stopping the agent and cleaning up...

✅ Tutorial 2 Complete!



# ================================================================
# SUMMARY OF WHAT WE LEARNED
# ================================================================

print("📚 WHAT WE LEARNED:")


print("=" * 40)

print("1. 🧠 **Built a working memory system")**

print("   - MemoryItem class for structured storage")

print("   - WorkingMemory class with capacity management")

print("   - Thread-safe memory operations")

print("   - Automatic cleanup when capacity is exceeded")

print()

print("**2. 🔄 Enhanced agent architecture")**

print("   - BaseAgent now includes memory capabilities")

print("   - remember() and recall() methods for easy access")

print("   - Memory metrics integrated with agent metrics")

print("   - Context retrieval for conversation continuity")

print()

print(**"3. 💬 Created a conversation-aware agent")**

print("   - ConversationAgent uses memory for context")

print("   - Context-sensitive response generation")

print("   - Memory of both incoming and outgoing messages")

print("   - Turn tracking and conversation flow")

print()

print(**"4. 📊 Added memory monitoring")**

print("   - Memory hit/miss ratio tracking")

print("   - Capacity utilization monitoring")

print("   - Age and access pattern analysis")

print("   - Integration with existing metrics system")

print()


 =============================================================================
# COMMON ERRORS AND SOLUTIONS
# =============================================================================

print("⚠️  COMMON ERRORS AND SOLUTIONS:")

print("=" * 40)

print("1. 🐛 'Memory capacity exceeded' warnings")

print("   Problem: Agent trying to store more than capacity allows")

print("   Solution: This is normal! Old memories are automatically removed")

print("   Solution: Increase capacity if needed: WorkingMemory(capacity=20)")

print()

print("2. 🐛 Thread safety issues with memory")

print("   Problem: Multiple threads accessing memory simultaneously")

print("   Solution: Already handled with threading.Lock() in WorkingMemory")

print("   Solution: Always use remember() and recall() methods")

print()

print("3. 🐛 Memory items not being found")

print("   Problem: Query doesn't match stored content")

print("   Solution: recall() does simple text matching - be flexible")

print("   Solution: Store additional metadata for better searching")

print()

print("4. 🐛 High memory usage over time")

print("   Problem: Working memory growing without cleanup")

print("   Solution: Already handled by capacity limits")

print("   Solution: Monitor memory_stats for utilization trends")

print()

print("5. 🐛 Context not being maintained properly")

print("   Problem: Important memories being discarded")

print("   Solution: Use higher importance values for key information")

print("   Solution: Increase memory capacity for longer conversations")

print()

print("🎉 Ready for Tutorial 3: Agent Communication!")

print("   Next we'll teach agents to talk to each other...")

---

🔒 **INTELLECTUAL PROPERTY & LICENSE NOTICE**

This tutorial and its contents — including code, architecture, narrative examples, and educational structure — are the intellectual property of **Shalini Ananda, PhD** and part of the **Neuron Framework** under a **Modified MIT License with Attribution**.

- Commercial use, redistribution, or derivative works **must** include clear and visible attribution to the original author.
- Use in products, consulting engagements, or educational materials **must reference this repository and author name.**
- Removal of author credit or misrepresentation of origin constitutes **a violation of the license and may trigger legal action.**
- You may **not white-label, obfuscate, or rebrand** this work without explicit, written permission.

Use of this tutorial in Colab or any other platform implies agreement with these terms.

📘 **License**: [LICENSE.md](../LICENSE.md)  
📌 **Notice**: [NOTICE.md](../NOTICE.md)  
🧠 **Author**: [Shalini Ananda, PhD](https://github.com/ShaliniAnandaPhD)

---

